In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split

%matplotlib inline

In [2]:
def sigmoid(z):
    """
    Compute the sigmoid of z

    Arguments:
    z -- A scalar or numpy array of any size.

    Return:
    s -- sigmoid(z)
    """
    s = 1/(1+(np.exp(-z)))
    
    return s

In [3]:
def layer_sizes(X, Y):
    """
    Arguments:
    X -- input dataset of shape (input size, number of examples)
    Y -- labels of shape (output size, number of examples)
    
    Returns:
    n_x -- the size of the input layer
    n_h -- the size of the hidden layer
    n_y -- the size of the output layer
    """
    n_x = len(X) # size of input layer
    n_h = 4
    n_y = len(Y) # size of output layer
    return (n_x, n_h, n_y)

In [4]:
def initialize_parameters(n_x, n_h, n_y):
    """
    Argument:
    n_x -- size of the input layer
    n_h -- size of the hidden layer
    n_y -- size of the output layer
    
    Returns:
    params -- python dictionary containing your parameters:
                    W1 -- weight matrix of shape (n_h, n_x)
                    b1 -- bias vector of shape (n_h, 1)
                    W2 -- weight matrix of shape (n_y, n_h)
                    b2 -- bias vector of shape (n_y, 1)
    """
    
    np.random.seed(2) # we set up a seed so that your output matches ours although the initialization is random.
    

    W1 = np.random.randn(n_h,n_x) * 0.01
    b1 = np.zeros((n_h,1))
    W2 = np.random.randn(n_y,n_h) * 0.01
    b2 = np.zeros((n_y,1))
    
    assert (W1.shape == (n_h, n_x))
    assert (b1.shape == (n_h, 1))
    assert (W2.shape == (n_y, n_h))
    assert (b2.shape == (n_y, 1))
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

In [5]:
def forward_propagation(X, parameters):
    """
    Argument:
    X -- input data of size (n_x, m)
    parameters -- python dictionary containing your parameters (output of initialization function)
    
    Returns:
    A2 -- The sigmoid output of the second activation
    cache -- a dictionary containing "Z1", "A1", "Z2" and "A2"
    """
    # Retrieve each parameter from the dictionary "parameters"
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    
    # Implement Forward Propagation to calculate A2 (probabilities)
    Z1 = np.dot(W1,X) + b1
    A1 = np.tanh(Z1)
    Z2 = np.dot(W2,A1) + b2
    A2 = sigmoid(Z2)
    
    assert(A2.shape == (1, X.shape[1]))
    
    cache = {"Z1": Z1,
             "A1": A1,
             "Z2": Z2,
             "A2": A2}
    
    return A2, cache

In [6]:
def compute_cost(A2, Y, parameters):
    """
    Computes the cross-entropy cost given in equation (13)
    
    Arguments:
    A2 -- The sigmoid output of the second activation, of shape (1, number of examples)
    Y -- "true" labels vector of shape (1, number of examples)
    parameters -- python dictionary containing your parameters W1, b1, W2 and b2
    
    Returns:
    cost -- cross-entropy cost given equation (13)
    """
    
    m = Y.shape[1] # number of example

    # Compute the cross-entropy cost
    logprobs = np.multiply(np.log(A2),Y) + np.multiply(1 - Y, np.log(1 - A2))
    cost = - np.sum(logprobs) / len(Y[0])
    
    cost = np.squeeze(cost)     # makes sure cost is the dimension we expect. 
                                # E.g., turns [[17]] into 17 
    assert(isinstance(cost, float))
    
    return cost

In [7]:
def backward_propagation(parameters, cache, X, Y):
    """
    Implement the backward propagation using the instructions above.
    
    Arguments:
    parameters -- python dictionary containing our parameters 
    cache -- a dictionary containing "Z1", "A1", "Z2" and "A2".
    X -- input data of shape (2, number of examples)
    Y -- "true" labels vector of shape (1, number of examples)
    
    Returns:
    grads -- python dictionary containing your gradients with respect to different parameters
    """
    m = X.shape[1]
    
    # First, retrieve W1 and W2 from the dictionary "parameters".
    W1 = parameters['W1']
    W2 = parameters['W2']
        
    # Retrieve also A1 and A2 from dictionary "cache".
    A1 = cache['A1']
    A2 = cache['A2']
    
    # Backward propagation: calculate dW1, db1, dW2, db2. 
    dZ2 = A2 - Y
    dW2 = (np.dot(dZ2,A1.T))/len(Y[0])
    db2 = (np.sum(dZ2, axis=1, keepdims=True))/len(Y[0])
    dZ1 = np.multiply( np.dot(W2.T,dZ2), 1 - np.power(A1, 2))
    dW1 = (np.dot(dZ1,X.T))/len(Y[0])
    db1 = (np.sum(dZ1, axis=1, keepdims=True))/len(Y[0])
    
    grads = {"dW1": dW1,
             "db1": db1,
             "dW2": dW2,
             "db2": db2}
    
    return grads

In [8]:
def update_parameters(parameters, grads, learning_rate = 1.2):
    """
    Updates parameters using the gradient descent update rule given above
    
    Arguments:
    parameters -- python dictionary containing your parameters 
    grads -- python dictionary containing your gradients 
    
    Returns:
    parameters -- python dictionary containing your updated parameters 
    """
    # Retrieve each parameter from the dictionary "parameters"
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    
    # Retrieve each gradient from the dictionary "grads"
    dW1 = grads["dW1"]
    db1 = grads["db1"]
    dW2 = grads["dW2"]
    db2 = grads["db2"]
    
    # Update rule for each parameter
    W1 = W1 - learning_rate*dW1
    b1 = b1 - learning_rate*db1
    W2 = W2 - learning_rate*dW2
    b2 = b2 - learning_rate*db2
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

In [9]:
def nn_model(X, Y, n_h, learning_rate, num_iterations = 10000, print_cost=False):
    """
    Arguments:
    X -- dataset of shape (2, number of examples)
    Y -- labels of shape (1, number of examples)
    n_h -- size of the hidden layer
    num_iterations -- Number of iterations in gradient descent loop
    print_cost -- if True, print the cost every 1000 iterations
    
    Returns:
    parameters -- parameters learnt by the model. They can then be used to predict.
    """
    
    np.random.seed(3)
    n_x = layer_sizes(X, Y)[0]
    n_y = layer_sizes(X, Y)[2]
    
    # Initialize parameters, then retrieve W1, b1, W2, b2. Inputs: "n_x, n_h, n_y". Outputs = "W1, b1, W2, b2, parameters".
    parameters = initialize_parameters(n_x, n_h, n_y)
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    
    # Loop (gradient descent)

    for i in range(0, num_iterations):
        
        # Forward propagation. Inputs: "X, parameters". Outputs: "A2, cache".
        A2, cache = forward_propagation(X, parameters)
        
        # Cost function. Inputs: "A2, Y, parameters". Outputs: "cost".
        cost = compute_cost(A2, Y, parameters)
 
        # Backpropagation. Inputs: "parameters, cache, X, Y". Outputs: "grads".
        grads = backward_propagation(parameters, cache, X, Y)
 
        # Gradient descent parameter update. Inputs: "parameters, grads". Outputs: "parameters".
        parameters = update_parameters(parameters, grads, learning_rate)
        
        
        # Print the cost every 1000 iterations
        if print_cost and i % 1000 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))

    return parameters

In [10]:
def predict(parameters, X):
    """
    Using the learned parameters, predicts a class for each example in X
    
    Arguments:
    parameters -- python dictionary containing your parameters 
    X -- input data of size (n_x, m)
    
    Returns
    predictions -- vector of predictions of our model (red: 0 / blue: 1)
    """
    
    # Computes probabilities using forward propagation, and classifies to 0/1 using 0.5 as the threshold.
    A2, cache = forward_propagation(X, parameters)
    predictions = A2 > 0.5
    
    return predictions

In [11]:
#dataset with two features

iris = datasets.load_iris()
x = iris.data
y = ((iris.target != 0) * 1)  # convert the target's dataset into binary

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.3)

m_train = len(x_train)
m_test = len(x_test)
num_n = x_train.shape[1]
numtest_n = x_test.shape[1]

y_train = y_train.reshape(m_train,1)
y_test = y_test.reshape(m_test,1)

print ("Number of training examples: m_train = " + str(m_train))
print ("Number of testing examples: m_test = " + str(m_test))
print ("number of features of each example: num_n = " + str(num_n))
print("shape of x_train:",x_train.shape)
print("shape of y_train:",y_train.shape)
print("shape of x_test:",x_test.shape)
print("shape of y_test:",y_test.shape)

Number of training examples: m_train = 105
Number of testing examples: m_test = 45
number of features of each example: num_n = 4
shape of x_train: (105, 4)
shape of y_train: (105, 1)
shape of x_test: (45, 4)
shape of y_test: (45, 1)


In [29]:
parameters = nn_model(x_train.T, y_train.T, n_h = 4,  learning_rate=0.1, num_iterations = 30000, print_cost=True)

predictions = predict(parameters, x_train.T)
print ('Train Accuracy: %d' % float((np.dot(y_train.T,predictions.T) + np.dot(1-y_train.T,1-predictions.T))/float(y_train.T.size)*100) + '%')

predictions = predict(parameters, x_test.T)
print ('Test Accuracy: %d' % float((np.dot(y_test.T,predictions.T) + np.dot(1-y_test.T,1-predictions.T))/float(y_test.T.size)*100) + '%')

Cost after iteration 0: 0.693503
Cost after iteration 1000: 0.003272
Cost after iteration 2000: 0.001505
Cost after iteration 3000: 0.000971
Cost after iteration 4000: 0.000715
Cost after iteration 5000: 0.000565
Cost after iteration 6000: 0.000467
Cost after iteration 7000: 0.000397
Cost after iteration 8000: 0.000346
Cost after iteration 9000: 0.000306
Cost after iteration 10000: 0.000274
Cost after iteration 11000: 0.000249
Cost after iteration 12000: 0.000227
Cost after iteration 13000: 0.000209
Cost after iteration 14000: 0.000194
Cost after iteration 15000: 0.000181
Cost after iteration 16000: 0.000169
Cost after iteration 17000: 0.000159
Cost after iteration 18000: 0.000150
Cost after iteration 19000: 0.000142
Cost after iteration 20000: 0.000134
Cost after iteration 21000: 0.000128
Cost after iteration 22000: 0.000122
Cost after iteration 23000: 0.000117
Cost after iteration 24000: 0.000112
Cost after iteration 25000: 0.000107
Cost after iteration 26000: 0.000103
Cost after ite

In [18]:
#Another way to test
print("Test Accuracy:",100 - np.mean(np.abs(predictions - y_test.T)) * 100,"%")

Test Accuracy: 100.0 %


In [30]:
#testing with different number of neurons and learning-rate = 1.1
hidden_layer_sizes = [1, 2, 3, 4, 5, 20, 50]
for n_h in hidden_layer_sizes:
    print(n_h)
    print('Hidden Layer of size %d' % n_h,":")
    parameters = nn_model(x_train.T, y_train.T, n_h = n_h,  learning_rate=1.1, num_iterations = 30000, print_cost=False)

    predictions = predict(parameters, x_train.T)
    print ('Train Accuracy: %d' % float((np.dot(y_train.T,predictions.T) + np.dot(1-y_train.T,1-predictions.T))/float(y_train.T.size)*100) + '%')

    predictions = predict(parameters, x_test.T)
    print ('Test Accuracy: %d' % float((np.dot(y_test.T,predictions.T) + np.dot(1-y_test.T,1-predictions.T))/float(y_test.T.size)*100) + '%')
    print("")

1
Hidden Layer of size 1 :
Train Accuracy: 100%
Test Accuracy: 100%

2
Hidden Layer of size 2 :
Train Accuracy: 100%
Test Accuracy: 100%

3
Hidden Layer of size 3 :
Train Accuracy: 100%
Test Accuracy: 100%

4
Hidden Layer of size 4 :
Train Accuracy: 100%
Test Accuracy: 100%

5
Hidden Layer of size 5 :
Train Accuracy: 100%
Test Accuracy: 100%

20
Hidden Layer of size 20 :
Train Accuracy: 71%
Test Accuracy: 55%

50
Hidden Layer of size 50 :


/home/dan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:18: RuntimeWarning: divide by zero encountered in log
/home/dan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in multiply


Train Accuracy: 28%
Test Accuracy: 44%

